This notebook provides a comprehensive guide for setting up and managing the development environment for our AI agent system. It includes profile management, Docker configuration, networking, and tool installation.

We'll implement a profile management system that handles different development environments and their dependencies.

In [ ]:
import yaml
import json
from typing import Dict, List, Optional
from dataclasses import dataclass
from pathlib import Path

@dataclass
class Profile:
    name: str
    description: str
    dependencies: List[str]
    packages: List[str]
    env_vars: Dict[str, str]
    
class ProfileManager:
    def __init__(self, config_path: Path):
        self.config_path = config_path
        self.profiles = self._load_profiles()
    
    def _load_profiles(self) -> Dict[str, Profile]:
        if not self.config_path.exists():
            return {}
        
        with open(self.config_path) as f:
            data = yaml.safe_load(f)
            return {
                name: Profile(
                    name=name,
                    description=info.get('description', ''),
                    dependencies=info.get('dependencies', []),
                    packages=info.get('packages', []),
                    env_vars=info.get('env_vars', {})
                )
                for name, info in data.items()
            }
    
    def get_profile(self, name: str) -> Optional[Profile]:
        return self.profiles.get(name)
    
    def list_profiles(self) -> List[Profile]:
        return list(self.profiles.values())
    
    def get_dependencies(self, name: str) -> List[str]:
        profile = self.get_profile(name)
        if not profile:
            return []
            
        deps = set(profile.dependencies)
        for dep in profile.dependencies:
            deps.update(self.get_dependencies(dep))
        return list(deps)

Configure Docker containers with proper UID/GID mapping and volume mounts.

In [ ]:
import os
import docker
from docker.types import Mount

class DevContainer:
    def __init__(self, project_name: str, profile: Profile):
        self.client = docker.from_env()
        self.project_name = project_name
        self.profile = profile
        
    def create_container(self, workspace_path: Path):
        # Get host user UID/GID
        uid = os.getuid()
        gid = os.getgid()
        
        # Configure volume mounts
        mounts = [
            Mount(
                target="/workspace",
                source=str(workspace_path.absolute()),
                type="bind"
            ),
            Mount(
                target="/home/dev/.config",
                source=str(workspace_path / ".config"),
                type="bind"
            )
        ]
        
        # Create container
        container = self.client.containers.run(
            f"dev-env:{self.profile.name}",
            command="zsh",
            detach=True,
            tty=True,
            mounts=mounts,
            environment={
                "HOST_UID": str(uid),
                "HOST_GID": str(gid),
                **self.profile.env_vars
            },
            name=f"{self.project_name}-dev"
        )
        
        return container

Set up network firewall rules and project-specific allowlists.

In [ ]:
from dataclasses import dataclass
from typing import List

@dataclass
class NetworkRule:
    port: int
    protocol: str
    source: str
    description: str

class NetworkManager:
    def __init__(self, project_name: str):
        self.project_name = project_name
        self.rules: List[NetworkRule] = []
        
    def add_rule(self, rule: NetworkRule):
        self.rules.append(rule)
        
        # Apply rule to Docker container
        container_name = f"{self.project_name}-dev"
        cmd = [
            "ufw",
            "allow",
            "from", rule.source,
            "to", "any",
            "port", str(rule.port),
            "proto", rule.protocol,
            "comment", f"{self.project_name}: {rule.description}"
        ]
        
        container = docker.from_env().containers.get(container_name)
        container.exec_run(cmd)
    
    def list_rules(self) -> List[NetworkRule]:
        return self.rules
    
    def remove_rule(self, port: int, protocol: str):
        self.rules = [r for r in self.rules 
                     if not (r.port == port and r.protocol == protocol)]
        
        # Remove rule from Docker container
        container_name = f"{self.project_name}-dev"
        cmd = ["ufw", "delete", "allow", f"{port}/{protocol}"]
        
        container = docker.from_env().containers.get(container_name)
        container.exec_run(cmd)

Implement isolation mechanisms for separate Docker images, authentication states, and configurations.

In [ ]:
from dataclasses import dataclass
import shutil

@dataclass
class ProjectConfig:
    name: str
    profiles: List[str]
    workspace_path: Path
    data_path: Path

class ProjectManager:
    def __init__(self, base_path: Path):
        self.base_path = base_path
        self.projects: Dict[str, ProjectConfig] = {}
        
    def create_project(self, name: str, profiles: List[str]) -> ProjectConfig:
        workspace_path = self.base_path / name / "workspace"
        data_path = self.base_path / name / "data"
        
        # Create directory structure
        workspace_path.mkdir(parents=True, exist_ok=True)
        data_path.mkdir(parents=True, exist_ok=True)
        
        config = ProjectConfig(
            name=name,
            profiles=profiles,
            workspace_path=workspace_path,
            data_path=data_path
        )
        
        self.projects[name] = config
        return config
    
    def delete_project(self, name: str):
        if name not in self.projects:
            return
            
        project = self.projects[name]
        
        # Stop and remove container
        try:
            container = docker.from_env().containers.get(f"{name}-dev")
            container.stop()
            container.remove()
        except:
            pass
            
        # Remove project directory
        shutil.rmtree(self.base_path / name)
        del self.projects[name]

Script the installation of common development tools.

In [ ]:
class ToolInstaller:
    @staticmethod
    def install_github_cli():
        commands = [
            "curl -fsSL https://cli.github.com/packages/githubcli-archive-keyring.gpg | sudo dd of=/usr/share/keyrings/githubcli-archive-keyring.gpg",
            "echo 'deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/githubcli-archive-keyring.gpg] https://cli.github.com/packages stable main' | sudo tee /etc/apt/sources.list.d/github-cli.list > /dev/null",
            "sudo apt update",
            "sudo apt install gh"
        ]
        return commands
    
    @staticmethod
    def install_delta():
        commands = [
            "wget https://github.com/dandavison/delta/releases/download/0.17.0/git-delta_0.17.0_amd64.deb",
            "sudo dpkg -i git-delta_0.17.0_amd64.deb",
            "rm git-delta_0.17.0_amd64.deb"
        ]
        return commands
    
    @staticmethod
    def install_uv():
        commands = [
            "curl -LsSf https://astral.sh/uv/install.sh | sh"
        ]
        return commands
    
    @staticmethod
    def install_nala():
        commands = [
            "sudo apt update",
            "sudo apt install nala"
        ]
        return commands

Configure ZSH with oh-my-zsh and additional utilities.

In [ ]:
class ShellConfigurator:
    @staticmethod
    def setup_zsh():
        commands = [
            # Install ZSH
            "sudo apt install zsh",
            
            # Install Oh My ZSH
            "sh -c '$(curl -fsSL https://raw.githubusercontent.com/ohmyzsh/ohmyzsh/master/tools/install.sh)'",
            
            # Install Powerline fonts
            "sudo apt install fonts-powerline",
            
            # Install fzf
            "git clone --depth 1 https://github.com/junegunn/fzf.git ~/.fzf",
            "~/.fzf/install --all"
        ]
        return commands
    
    @staticmethod
    def configure_zsh(theme: str = "agnoster"):
        config = f"""
# Oh My ZSH Configuration
export ZSH="$HOME/.oh-my-zsh"
ZSH_THEME="{theme}"

# Plugins
plugins=(
    git
    docker
    docker-compose
    fzf
    zsh-autosuggestions
    zsh-syntax-highlighting
)

source $ZSH/oh-my-zsh.sh

# FZF Configuration
[ -f ~/.fzf.zsh ] && source ~/.fzf.zsh
"""
        return config

Implement mechanisms for persisting various states and configurations.

In [ ]:
import sqlite3
from typing import Any, Dict

class StateManager:
    def __init__(self, project_path: Path):
        self.db_path = project_path / "state.db"
        self._init_db()
    
    def _init_db(self):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # Create tables
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS auth_state (
                service TEXT PRIMARY KEY,
                token TEXT,
                expiry TEXT
            )
        """)
        
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS shell_history (
                timestamp TEXT,
                command TEXT,
                exit_code INTEGER
            )
        """)
        
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS tool_config (
                tool TEXT PRIMARY KEY,
                config TEXT
            )
        """)
        
        conn.commit()
        conn.close()
    
    def save_auth_state(self, service: str, token: str, expiry: str):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute(
            "INSERT OR REPLACE INTO auth_state VALUES (?, ?, ?)",
            (service, token, expiry)
        )
        
        conn.commit()
        conn.close()
    
    def add_to_history(self, command: str, exit_code: int):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute(
            "INSERT INTO shell_history VALUES (datetime('now'), ?, ?)",
            (command, exit_code)
        )
        
        conn.commit()
        conn.close()
    
    def save_tool_config(self, tool: str, config: Dict[str, Any]):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute(
            "INSERT OR REPLACE INTO tool_config VALUES (?, ?)",
            (tool, json.dumps(config))
        )
        
        conn.commit()
        conn.close()

Here's a complete example of setting up a development environment for a Python project:

In [ ]:
# Initialize managers
profile_manager = ProfileManager(Path("/workspaces/Asynchronous/config/profiles.yml"))
project_manager = ProjectManager(Path("/workspaces/Asynchronous/projects"))

# Create a new Python project
python_profile = profile_manager.get_profile("python")
project = project_manager.create_project("my-python-project", ["python", "core"])

# Create development container
container = DevContainer("my-python-project", python_profile)
container.create_container(project.workspace_path)

# Configure networking
network = NetworkManager("my-python-project")
network.add_rule(NetworkRule(
    port=8000,
    protocol="tcp",
    source="127.0.0.1",
    description="Development server"
))

# Initialize state management
state = StateManager(project.data_path)

# Configure shell environment
shell = ShellConfigurator()
shell_commands = shell.setup_zsh()
zsh_config = shell.configure_zsh()

print("Development environment setup complete!")